In [58]:
from collections import Counter, defaultdict
from transformers import BertTokenizer,AutoModel, AutoTokenizer
import sentencepiece as spm
import pandas as pd
import heapq
import re
from typing import Iterable
import numpy as np
import matplotlib.pyplot as plt
import pickle

from tokenizers import Tokenizer, models, pre_tokenizers, trainers

import nltk
from nltk.tokenize import WhitespaceTokenizer

In [2]:
# ALL these List are prepared with the help of internet.
matras = set(['ા', 'િ', 'ી', 'ુ', 'ૂ', 'ે', 'ૈ', 'ો', 'ૌ', 'ં', 'ઃ', 'ૃ', 'ૅ', 'ૉ'])
vyanjans = set(['ક', 'ખ', 'ગ', 'ઘ', 'ઙ', 'ચ', 'છ', 'જ', 'ઝ', 'ઞ', 'ય', 'શ', 'ટ', 'ઠ', 'ડ', 'ઢ', 'ણ', 'ર', 'ષ', 'ત', 'થ', 'દ', 'ધ', 'ન', 'લ', 'સ', 'પ', 'ફ', 'બ', 'ભ', 'મ', 'વ', 'હ', 'ળ', 'ક્ષ', 'જ્ઞ']
)
dic = {
    'ા': 'આ', 'િ': 'ઇ', 'ી': 'ઈ', 'ુ': 'ઉ',
    'ૂ': 'ઊ', 'ે': 'એ', 'ૈ': 'ઐ', 'ો': 'ઓ', 'ૌ': 'ઔ',
    'ં': 'અં', 'ઃ': 'અ:', 'ૃ': 'ઋ', 'ૅ': 'ઍ', 'ૉ': 'ઑ'
}
# Making dictionary to map matras with their swar sound.
swars = set(['અ', 'આ', 'ઇ', 'ઈ', 'ઉ', 'ઊ', 'એ', 'ઐ', 'ઓ', 'ઔ', 'અં', 'અ:', 'ઋ', 'ઍ', 'ઑ'])
punctuations = set(['>्','0्', '1्', '"्', 'ळ्', '2्', '–्', 'ः्', '3्', '5्', '4्',
                    '््', '%्', '—्', '8्', '6्', '7्', '9्', 'ॅ्', 'a्', 'e्', '#्',
                    'i्', 'r्', 't्', '»्', 'o्', 'n्', 'd्', '०्', 's्', 'h्',
                    'l्', 'c्', 'm्', '\u200c्', '\u200b्', 'ï्', 'A्', 'p्',
                    '•्', 'b्', 'G्', 'B्', '&्', 'u्', '_्', '@्', 'M्', 'о्',
                    'f्', '·्', '$्', 'S्', 'g्', 'I्', 'а्', 'е्', 'P्', 'и्',
                    'R्', 'y्', 'k्', 'w्', 'T्', '�्', 'a', 'b', 'c', 'd', 'e',
                    'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r',
                    's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '0', '1', '2', '3', '4',
                    '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H',
                    'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U',
                    'V', 'W', 'X', 'Y', 'Z', '!', '"', '\\', '#', '\\', '$', '%', '\\',
                    '&', "'", '\\', '(', '\\', ')', '\\', '*', '\\', '+', ',', '\\', '-',
                    '\\', '.', '/', ':', ';', '<', '=', '>', '\\', '?', '@', '\\', '[', '\\',
                    '\\', '\\', ']', '\\', '^', '_', '`', '\\', '{', '\\', '|', '\\', '}', '\\',
                    '~', '‘', '’', '“', '”', '૰', '।', '॥'])# These are noise we just have to insure these does'nt occur and even if these occurs we just ignore them.

# We created set of the list to imporve the look_up time as in sets lookup time if of O(1).


In [32]:
def remove_trivial_prefix_suffix(input_string):
    # Define regular expression pattern to match any sequence of characters from the specified set
    pattern = r'^(?:CLS|SEP|[\W_]+)|[\W_]+$'

    # Use re.sub() to remove the matched sequences from the beginning and end of the string
    result = re.sub(pattern, '', input_string)

    # Remove extra spaces, ".", "!", "?", or any other symbol
    result = re.sub(r'\s+', ' ', result)  # Remove extra spaces
    result = re.sub(r'[^\w\s]', '', result)  # Remove symbols except word characters and spaces

    return result


In [59]:
def unicode(word):
    Corrected_word = ""
    l = len(word)
    for i in range(l):
        if word[i] in vyanjans:
            Corrected_word += ' ' +  word[i]+'्'
            if i <= l-2 and (word[i+1]=='्' or word[i+1] in punctuations) :   # IF this condition satisfied then we are doing nothing.
                pass
            elif i <= l-2 and (word[i+1] in vyanjans or word[i+1] in swars):
                Corrected_word += ' '+'અ'
            elif i <= l-2 and word[i+1] in matras:
                Corrected_word += ' '+ dic[word[i+1]]
            elif i == l - 1:
                Corrected_word += ' ' + 'અ'
        elif word[i] in swars:
            Corrected_word += ' '+ word[i]
    return Corrected_word

In [40]:
#syllable_count
def PurnaVyanjan(token,i):
    if  i < len(token) - 1:
        if token[i+1] in vyanjans:
            return token[i]
        elif token[i+1] in swars or token[i+1] in matras:
            return token[i:i+2]
        elif token[i+1] == '्':
            return token[i:i+2] + PurnaVyanjan(token,i+2)
    elif i >= len(token):
        return ""
    return token[i]

# **Question 4**

In [62]:
# Read file and store it the gujarati_corpus
#Given file will be used to perform serveral tokenization models
try:
    with open('gu_100.txt', 'r', encoding='utf-8') as file:
        gujarati_corpus = file.readlines()
except UnicodeDecodeError:
    print("Error: Unable to decode file with UTF-8 encoding. Trying different encoding...")
    with open('gu_100.txt', 'r', encoding='latin-1') as file:
        gujarati_corpus = file.readlines()

In [63]:
# Function arg=(list of list), and returns a signle list
def flatten_list(list_of_lists):
    flattened_list = []
    for sublist in list_of_lists:
        flattened_list.extend(sublist)
    return flattened_list

### **Unigram Freq of tokens, char, syllabals**

In [64]:
# Functions creates a list containing tokens and there frequency
# Argument passed = List of tokens
# Prints Tokens with their frequency in descending order
def freqToken(tokens):
    token_counter = Counter(tokens)

  # Sort tokens by frequency in descending order
    sorted_tokens = sorted(token_counter.items(), key=lambda x: x[1], reverse=True)

    # Print sorted tokens
    print("Tokens with their frequency")
    for token, freq in sorted_tokens:
        print(f"({token}, {freq})", end="  ")

In [65]:
# Functions creates a list containing char and there frequency
# Argument passed = List of tokens
# Prints characters with their frequency in descending order
def freqChar(tokenized_corpus):
  # for tokenized_corpus in tokens:
    char_list = []
    for i in tokenized_corpus:
        for j in unicode(i).split():   #unicode_correction of tokens and counting characters.Unicode(i) will return corrected word.
            char_list.append(j)
    char_dict_bpe = Counter(char_list)
    freq_char_bpe = heapq.nlargest(len(char_dict_bpe.keys()), char_dict_bpe, key = char_dict_bpe.get)  # freq_chars will store the character with decreasing order of their frequency.
    print("Characters with their frequency")
    for i in freq_char_bpe:
      print(f"({i}, {char_dict_bpe[i]})", end="  ")
        # print(i,char_dict_bpe[i])

In [66]:
# Functions creates a list containing syllables and there frequency
# Argument passed = List of tokens
# Prints syllables with their frequency in descending order
def freqSyllable(tokenized_corpus):
  # for tokenized_corpus in tokens:
    syllables_list = []
    for token in tokenized_corpus:
        i=0
        while i < len(token):
            if token[i] in vyanjans:
                string = PurnaVyanjan(token,i)   # calling purnavyanjan function.
                syllables_list.append(string)
                i = i + len(string)-1
            elif token[i] in swars:
                syllables_list.append(token[i])
            i += 1
    syllables_dict_bpe = Counter(syllables_list)

    freq_syllable_bpe = heapq.nlargest(len(syllables_dict_bpe.keys()), syllables_dict_bpe, key = syllables_dict_bpe.get) # freq_syllables_bpe will store the syllables with decreasing order of their frequency.
    print("Syllables with their frequency")
    for i in freq_syllable_bpe:
      print(f"({i}, {syllables_dict_bpe[i]})", end="  ")
        # print(i,syllables_dict_bpe[i])

# (1) IndicBERT

In [69]:
try:
    with open('gu_100.txt', 'r', encoding='utf-8') as file:
        gujarati_corpus = file.readlines()
except UnicodeDecodeError:
    print("Error: Unable to decode file with UTF-8 encoding. Trying different encoding...")
    with open('gu_100.txt', 'r', encoding='latin-1') as file:
        gujarati_corpus = file.readlines()

In [70]:
def IndicBert_Model(gujarati_corpus,length):
  IndicBERT_tokenizer = AutoTokenizer.from_pretrained('ai4bharat/indic-bert', keep_accents=True,max_length = length)
  tokenized_corpus = []
  for line in gujarati_corpus:
          cleaned_tokens = []
          tokens = IndicBERT_tokenizer.tokenize(line.strip())
          cleaned_tokens = [remove_trivial_prefix_suffix(token) for token in tokens]
          # for token in cleaned_tokens:
          tokenized_corpus.append(cleaned_tokens)
#   print(tokenized_corpus)
  return tokenized_corpus

In [72]:
indicbert1k = flatten_list(IndicBert_Model(gujarati_corpus,1000))
# indicbert1k

In [73]:
freqToken(indicbert1k)

Tokens with their frequency
(, 1135778)  (ન, 484906)  (છ, 342219)  (અન, 155468)  (મ, 149534)  (ત, 144451)  (ક, 134910)  (જ, 118447)  (હત, 103087)  (મટ, 97596)  (આ, 91258)  (કર, 88531)  (પર, 74774)  (પણ, 64286)  (એક, 63498)  (સ, 62234)  (એ, 59252)  (થ, 58623)  (સથ, 54978)  (ય, 50988)  (લ, 48198)  (ર, 47943)  (આવ, 45530)  (વ, 43498)  (તન, 42567)  (તમ, 42240)  (શ, 38002)  (બ, 37277)  (ડ, 35047)  (શક, 32751)  (નથ, 31824)  (કરવ, 30650)  (ગ, 30535)  (થય, 27276)  (ટ, 26936)  (કરય, 26640)  (પ, 26518)  (તમન, 25458)  (હ, 25401)  (તમર, 23297)  (હય, 23083)  (વધ, 23052)  (રત, 23016)  (પરત, 22956)  (કરવમ, 20770)  (કરણ, 20188)  (જવ, 19920)  (પછ, 19289)  (દવર, 19014)  (દ, 18921)  (રહય, 18912)  (મન, 18055)  (ઓ, 17640)  (પતન, 17636)  (આવય, 17578)  (કમ, 17568)  (દર, 17361)  (બન, 17307)  (અથવ, 17189)  (રહ, 17089)  (તયર, 16835)  (બધ, 16643)  (કરત, 16618)  (ચ, 16496)  (જન, 16418)  (મર, 16257)  (લક, 16068)  (સર, 15971)  (સમ, 15751)  (કઈ, 15695)  (મળ, 15645)  (સધ, 15570)  (હવ, 15394)  (ઝ, 15218)  (આપ, 14894) 

In [74]:
freqChar(indicbert1k)


Characters with their frequency
(અ, 18559265)  (ર्, 2128526)  (ન्, 1734112)  (ક्, 1386555)  (મ्, 1324843)  (ત्, 1207724)  (વ्, 1149790)  (સ्, 976491)  (પ्, 867048)  (ય्, 745130)  (લ्, 684911)  (જ्, 585042)  (ટ्, 520557)  (હ्, 516880)  (દ्, 427491)  (છ्, 412209)  (ગ्, 396930)  (બ्, 379101)  (શ्, 374717)  (થ्, 368405)  (આ, 331134)  (ણ्, 310058)  (ડ्, 308070)  (એ, 250862)  (ધ्, 239054)  (ચ्, 224280)  (ખ्, 182786)  (ભ्, 168822)  (ળ्, 158447)  (ફ्, 150791)  (ઓ, 136048)  (ઈ, 123055)  (ષ्, 114074)  (ઇ, 101041)  (ઉ, 99613)  (ઘ्, 54607)  (ઝ्, 50170)  (ઠ्, 40439)  (ઢ्, 11663)  (ઊ, 8797)  (ઞ्, 4933)  (ઑ, 2580)  (ઔ, 1389)  (ઍ, 1271)  (ઋ, 919)  (ઐ, 913)  (ઙ्, 113)  

In [75]:
freqSyllable(indicbert1k)

Syllables with their frequency
(ર, 2104613)  (ન, 1727303)  (ક, 1352825)  (મ, 1320043)  (ત, 1181419)  (વ, 1143881)  (સ, 964194)  (પ, 859812)  (ય, 736584)  (લ, 662616)  (જ, 559113)  (ટ, 515285)  (હ, 506812)  (દ, 423380)  (છ, 408306)  (ગ, 388505)  (શ, 374036)  (બ, 370828)  (અ, 353150)  (થ, 345580)  (આ, 323223)  (ણ, 307178)  (ડ, 301214)  (ધ, 237197)  (એ, 223328)  (ચ, 222786)  (ખ, 181230)  (ભ, 157640)  (ળ, 155781)  (ફ, 146100)  (ષ, 112329)  (ઉ, 86979)  (ઓ, 56211)  (ઘ, 54349)  (ઝ, 48218)  (ઇ, 45152)  (ઠ, 40107)  (ઈ, 29745)  (તઓ, 25883)  (કઈ, 21490)  (જઈ, 14169)  (થઈ, 12412)  (ઢ, 11663)  (લઈ, 9722)  (રઓ, 9279)  (ઊ, 8797)  (લઇ, 8238)  (ભઈ, 7275)  (થઇ, 6409)  (કઇ, 5527)  (હઈ, 5343)  (ઞ, 4933)  (જઓ, 4608)  (જઇ, 4260)  (નઓ, 4040)  (થઓ, 4004)  (ભઇ, 3907)  (લઓ, 3738)  (બઈ, 3733)  (ગઈ, 3704)  (છએ, 3702)  (રઆ, 3591)  (યઓ, 3414)  (રઇ, 3382)  (રએ, 3361)  (સઓ, 3225)  (સઇ, 3124)  (દઓ, 3052)  (હઇ, 3039)  (ડઓ, 3037)  (યએ, 2986)  (કઉ, 2821)  (ફઇ, 2659)  (પએ, 2647)  (રઈ, 2597)  (ઑ, 2580)  (સએ, 2421)  (ગઉ, 22

In [76]:
indicbert2k = flatten_list(IndicBert_Model(gujarati_corpus,2000))

In [77]:
freqToken(indicbert2k)


Tokens with their frequency
(, 1135778)  (ન, 484906)  (છ, 342219)  (અન, 155468)  (મ, 149534)  (ત, 144451)  (ક, 134910)  (જ, 118447)  (હત, 103087)  (મટ, 97596)  (આ, 91258)  (કર, 88531)  (પર, 74774)  (પણ, 64286)  (એક, 63498)  (સ, 62234)  (એ, 59252)  (થ, 58623)  (સથ, 54978)  (ય, 50988)  (લ, 48198)  (ર, 47943)  (આવ, 45530)  (વ, 43498)  (તન, 42567)  (તમ, 42240)  (શ, 38002)  (બ, 37277)  (ડ, 35047)  (શક, 32751)  (નથ, 31824)  (કરવ, 30650)  (ગ, 30535)  (થય, 27276)  (ટ, 26936)  (કરય, 26640)  (પ, 26518)  (તમન, 25458)  (હ, 25401)  (તમર, 23297)  (હય, 23083)  (વધ, 23052)  (રત, 23016)  (પરત, 22956)  (કરવમ, 20770)  (કરણ, 20188)  (જવ, 19920)  (પછ, 19289)  (દવર, 19014)  (દ, 18921)  (રહય, 18912)  (મન, 18055)  (ઓ, 17640)  (પતન, 17636)  (આવય, 17578)  (કમ, 17568)  (દર, 17361)  (બન, 17307)  (અથવ, 17189)  (રહ, 17089)  (તયર, 16835)  (બધ, 16643)  (કરત, 16618)  (ચ, 16496)  (જન, 16418)  (મર, 16257)  (લક, 16068)  (સર, 15971)  (સમ, 15751)  (કઈ, 15695)  (મળ, 15645)  (સધ, 15570)  (હવ, 15394)  (ઝ, 15218)  (આપ, 14894) 

In [3]:
#syllable_count
def PurnaVyanjan(token,i):
    if  i < len(token) - 1:
        if token[i+1] in vyanjans:
            return token[i]
        elif token[i+1] in swars or token[i+1] in matras:
            return token[i:i+2]
        elif token[i+1] == '्':
            return token[i:i+2] + PurnaVyanjan(token,i+2)
    elif i >= len(token):
        return ""
    return token[i]

In [6]:
# print(PurnaVyanjan())
i=0
tokenTemp="ઓગસ્ટ"
syllables_list=[]
while i < len(tokenTemp):
        if tokenTemp[i] in vyanjans:
            string = PurnaVyanjan(tokenTemp,i)
            syllables_list.append(string)
            i = i + len(string)-1
        elif tokenTemp[i] in swars:
            syllables_list.append(tokenTemp[i])
        i += 1

print(syllables_list)

['ઓ', 'ગ', 'સ', 'ટ']


In [7]:
freqChar(indicbert2k)


NameError: name 'freqChar' is not defined

In [79]:
freqSyllable(indicbert2k)

Syllables with their frequency
(ર, 2104613)  (ન, 1727303)  (ક, 1352825)  (મ, 1320043)  (ત, 1181419)  (વ, 1143881)  (સ, 964194)  (પ, 859812)  (ય, 736584)  (લ, 662616)  (જ, 559113)  (ટ, 515285)  (હ, 506812)  (દ, 423380)  (છ, 408306)  (ગ, 388505)  (શ, 374036)  (બ, 370828)  (અ, 353150)  (થ, 345580)  (આ, 323223)  (ણ, 307178)  (ડ, 301214)  (ધ, 237197)  (એ, 223328)  (ચ, 222786)  (ખ, 181230)  (ભ, 157640)  (ળ, 155781)  (ફ, 146100)  (ષ, 112329)  (ઉ, 86979)  (ઓ, 56211)  (ઘ, 54349)  (ઝ, 48218)  (ઇ, 45152)  (ઠ, 40107)  (ઈ, 29745)  (તઓ, 25883)  (કઈ, 21490)  (જઈ, 14169)  (થઈ, 12412)  (ઢ, 11663)  (લઈ, 9722)  (રઓ, 9279)  (ઊ, 8797)  (લઇ, 8238)  (ભઈ, 7275)  (થઇ, 6409)  (કઇ, 5527)  (હઈ, 5343)  (ઞ, 4933)  (જઓ, 4608)  (જઇ, 4260)  (નઓ, 4040)  (થઓ, 4004)  (ભઇ, 3907)  (લઓ, 3738)  (બઈ, 3733)  (ગઈ, 3704)  (છએ, 3702)  (રઆ, 3591)  (યઓ, 3414)  (રઇ, 3382)  (રએ, 3361)  (સઓ, 3225)  (સઇ, 3124)  (દઓ, 3052)  (હઇ, 3039)  (ડઓ, 3037)  (યએ, 2986)  (કઉ, 2821)  (ફઇ, 2659)  (પએ, 2647)  (રઈ, 2597)  (ઑ, 2580)  (સએ, 2421)  (ગઉ, 22

# **Question 5**

In [80]:
True_Positive = False_Positive = 0

Ground_truth = set([
    "તે", "જલ્દિ", "ફિલ્મ", "નિકમ્મા થી", "કમબેક કરવા", "જઇ રહિ છે",
    "વોટર-પ્યોરિફિકેશન", "ઉપરાંત પણ", "અનેક ગુણ", "ફટકડી", "ધરાવે છે",
    "એન્ટિક્રાઇસ્ટ", "આજે", "જીવંત છે",
    "ફોટોમાં",
    "નમ્ર હોસ્ક", "વિશે", "વધુ માહિતી",
    "ઉમાકાંત પટેલ", "શરીરથી", "સ્વસ્થ", "અને", "મનથી", "મજબૂત", "હોવા છતા પણ", "ફીટ થવું", "નિશ્ચિત નથી",
    "જો તમે", "સલ્ફરમાંથી", "સામયિક ટેબલને", "નીચે", "ખસેડો",
    "ઈન્ડિયન આઈડલ 11ના", "આગામી એપિસોડમાં", "ઉદિત નારાયણ અને અલકા યાજ્ઞિક", "મહેમાન બનશે",
    "રૂમ અને ભોજન",
    "પટ પટતી પાપણો",
    "એથી તેમણે", "રાજીનામું", "આપી દીધું",
    "મને", "એવું", "માનવામાં", "આવતું નથી", "કે", "તે", "ક્યારેય", "બન્યું છે", "કે", "ક્યારેય નહીં",
    "સત્તાની", "લાલચમાં કે", "અંગત કારણસર", "ધવલસિંહ ઝાલાએ", "દોઢ વર્ષના", "સમયગાળામાં", "તેમના", "મત વિસ્તારને", "નધણિયાત", "બનાવી", "મતદારો", "અને", "કોંગ્રેસના કાર્યકરો", "સાથે", "વિશ્વાસઘાત",  "કર્યો હોવાના", "સુર", "સાથે", "બાયડના", "કોંગ્રેસના", "હોદ્દેદારો", "અને", "કાર્યકરો", "મોટી સંખ્યામાં", "બાયડ બસ સ્ટેન્ડ નજીક", "એકઠા થઈ", "ધવલસિંહ ઝાલા", "ગદ્દાર છે ના", "સૂત્રોચ્ચાર સાથે", "ધવલસિંહ ઝાલા", "૧૫ કરોડમાં",
    'વેચાયા', 'અને', 'બાયડની', 'પ્રજાને', 'તમણે', 'છેતર્યા', 'કોટક સ્કુલ', 'રાજકોટ', 'તમારી', 'રાશિના', 'લોકોએ', 'હાલ', 'દુર્ઘટનાઓથી', 'સાવધાન', 'રહેવાની', 'જરૂર છે', 'તેમણે', 'ક્હ્યું કે', 'આ', 'લેખમાં', 'અમે', 'તમને', 'કહીશું કે', 'સ્વિમિંગને', 'વધુ રસપ્રદ', 'પ્રદેશ', 'ઇન્સ્યુલેટર છે', '39 વર્ષોનો', 'ગેઇલ', 'પૂરા', 'રંગમાં છે', 'રોગ', 'એચઆઇવી ચેપ', 'અથવા', 'ઓન્કોલોજી સાથે', 'સંકળાયેલ હોય', 'ઉપરના ભાગમાં', 'તમારે', 'અંડાકાર', '(હેડ)', 'રજૂ કરવું જોઈએ', 'મોટાભાગના', 'વિપક્ષી દળોએ', '23 મે બાદ', 'બેઠક', 'માટે', 'સંમતિ', 'દર્શાવી છે', 'પરંતુ', 'અમે', 'જેના કારણે', 'તેની', 'માઇલેજ', 'વર્તમાન વેગન આર ની જેમ', '20kmplની', 'આસપાસ જ', 'રહેશે', 'ચિહ્નો', 'અને', 'આ રોગ ના', 'લક્ષણો', 'લાક્ષાણિક ઉપચાર', 'દ્વારા', 'ઘટાડી', 'શકાય છે'
])

In [81]:
valid_inputs = [
    "તે જલ્દિ ફિલ્મ નિકમ્મા થી કમબેક કરવા જઇ રહિ છે.",
    "વોટર-પ્યોરિફિકેશન ઉપરાંત પણ અનેક ગુણ ફટકડી ધરાવે છે.",
    "એન્ટિક્રાઇસ્ટ આજે જીવંત છે?",
    "ફોટોમાં",
    "નમ્ર હોસ્ક વિશે વધુ માહિતી",
    "ઉમાકાંત પટેલ શરીરથી સ્વસ્થ અને મનથી મજબૂત હોવા છતા પણ ફીટ થવું નિશ્ચિત નથી!",
    "જો તમે સલ્ફરમાંથી સામયિક ટેબલને નીચે ખસેડો",
    "ઈન્ડિયન આઈડલ 11ના આગામી એપિસોડમાં ઉદિત નારાયણ અને અલકા યાજ્ઞિક મહેમાન બનશે.",
    "રૂમ અને ભોજન",
    "પટ પટતી પાપણો :",
    "એથી તેમણે રાજીનામું આપી દીધું.",
    "મને એવું માનવામાં આવતું નથી કે તે ક્યારેય બન્યું છે કે ક્યારેય નહીં.",
    "સત્તાની લાલચમાં કે અંગત કારણસર ધવલસિંહ ઝાલાએ દોઢ વર્ષના સમયગાળામાં તેમના મત વિસ્તારને નધણિયાત બનાવી મતદારો અને કોંગ્રેસના કાર્યકરો સાથે વિશ્વાસઘાત કર્યો હોવાના સુર સાથે બાયડના કોંગ્રેસના હોદ્દેદારો અને કાર્યકરો મોટી સંખ્યામાં બાયડ બસ સ્ટેન્ડ નજીક એકઠા થઈ ધવલસિંહ ઝાલા ગદ્દાર છે ના સૂત્રોચ્ચાર સાથે ધવલસિંહ ઝાલા ૧૫ કરોડમાં વેચાયા અને બાયડની પ્રજાને તમણે છેતર્યા",
    "કોટક સ્કુલ રાજકોટ",
    "તમારી રાશિના લોકોએ હાલ દુર્ઘટનાઓથી સાવધાન રહેવાની જરૂર છે.",
    "તેમણે ક્હ્યું કે",
    "આ લેખમાં અમે તમને કહીશું કે સ્વિમિંગને વધુ રસપ્રદ",
    "પ્રદેશ ઇન્સ્યુલેટર છે.",
    "39 વર્ષોનો ગેઇલ પૂરા રંગમાં છે.",
    "રોગ એચઆઇવી ચેપ અથવા ઓન્કોલોજી સાથે સંકળાયેલ હોય",
    "ઉપરના ભાગમાં તમારે અંડાકાર (હેડ) રજૂ કરવું જોઈએ.",
    "મોટાભાગના વિપક્ષી દળોએ 23 મે બાદ બેઠક માટે સંમતિ દર્શાવી છે.",
    "પરંતુ અમે",
    "જેના કારણે તેની માઇલેજ વર્તમાન વેગન આરની જેમ 20kmplની આસપાસ જ રહેશે.",
    "ચિહ્નો અને આ રોગ ના લક્ષણો લાક્ષાણિક ઉપચાર દ્વારા ઘટાડી શકાય છે."
]


In [82]:
len(valid_inputs)

25

In [83]:
def precision_matrix(tokenized_corpus):
  True_Positive=False_Positive=0
  for tokenList in tokenized_corpus:
    for token in tokenList[1:-1]:
      if remove_trivial_prefix_suffix(token) in Ground_truth:
          True_Positive += 1
      else:
          #print(new_word[_])  to check what are the wrong words generated by BPE.
          False_Positive += 1
  False_negative = len(Ground_truth) - True_Positive

  Precision = True_Positive / (True_Positive + False_Positive)
  Recall = True_Positive / (True_Positive + False_negative)
  F_Measure = (2 * Precision * Recall) / (Precision + Recall)

  print("precision->",Precision,"Recall->",Recall,"F_Measure->",F_Measure)
  print("Number of wrong Words Generated by given Model->",False_Positive)



In [84]:
precision_matrix(IndicBert_Model(valid_inputs, 1000))

precision-> 0.0035971223021582736 Recall-> 0.007246376811594203 F_Measure-> 0.004807692307692309
Number of wrong Words Generated by given Model-> 277


In [85]:
precision_matrix(IndicBert_Model(valid_inputs, 2000))

precision-> 0.0035971223021582736 Recall-> 0.007246376811594203 F_Measure-> 0.004807692307692309
Number of wrong Words Generated by given Model-> 277
